In [1]:
import os
import requests
import zipfile
import pandas as pd
from unidecode import unidecode

### Acesso aos dados

In [2]:
DOWNLOAD_URL = "https://web3.antaq.gov.br/ea/txt/"

anos = ["2021", "2022", "2023"] 
arquivos_desejados = ["Atracacao", "Carga", "CargaConteinerizada"]

Criação do diretório "../Bronze_Layer"

In [3]:
os.makedirs("../Bronze_Layer", exist_ok=True)

Download dos arquivos

In [4]:
def baixar_arquivos():
    for ano in anos:
        for arquivo_desejado in arquivos_desejados:
            salvar_caminho = os.path.join("../Bronze_Layer", f"{arquivo_desejado}_{ano}.zip")
            if os.path.exists(salvar_caminho):
                print(f"Arquivo {salvar_caminho} já existe. Pulando...")
                continue
            
            arquivo_url = f"{DOWNLOAD_URL}{ano}{arquivo_desejado}.zip"

            print(f"Baixando {arquivo_url}...")
            with open(salvar_caminho, "wb") as f:
                f.write(requests.get(arquivo_url).content)
            
            print(f"Salvo: {salvar_caminho}")


baixar_arquivos()

Baixando https://web3.antaq.gov.br/ea/txt/2021Atracacao.zip...
Salvo: ../Bronze_Layer\Atracacao_2021.zip
Baixando https://web3.antaq.gov.br/ea/txt/2021Carga.zip...
Salvo: ../Bronze_Layer\Carga_2021.zip
Baixando https://web3.antaq.gov.br/ea/txt/2021CargaConteinerizada.zip...
Salvo: ../Bronze_Layer\CargaConteinerizada_2021.zip
Baixando https://web3.antaq.gov.br/ea/txt/2022Atracacao.zip...
Salvo: ../Bronze_Layer\Atracacao_2022.zip
Baixando https://web3.antaq.gov.br/ea/txt/2022Carga.zip...
Salvo: ../Bronze_Layer\Carga_2022.zip
Baixando https://web3.antaq.gov.br/ea/txt/2022CargaConteinerizada.zip...
Salvo: ../Bronze_Layer\CargaConteinerizada_2022.zip
Baixando https://web3.antaq.gov.br/ea/txt/2023Atracacao.zip...
Salvo: ../Bronze_Layer\Atracacao_2023.zip
Baixando https://web3.antaq.gov.br/ea/txt/2023Carga.zip...
Salvo: ../Bronze_Layer\Carga_2023.zip
Baixando https://web3.antaq.gov.br/ea/txt/2023CargaConteinerizada.zip...
Salvo: ../Bronze_Layer\CargaConteinerizada_2023.zip


Descompactação dos arquivos extraídos

In [5]:
def descompactar_arquivos():
    for arquivo in os.listdir("../Bronze_Layer"):
        if arquivo.endswith(".zip"):
            caminho_zip = os.path.join("../Bronze_Layer", arquivo)
            destino = os.path.join("../Silver_Layer", arquivo.replace(".zip", ""))

            os.makedirs(destino, exist_ok=True)

            try:
                with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
                    zip_ref.extractall(destino)
                    print(f"Descompactado: {caminho_zip} -> {destino}")
            except zipfile.BadZipFile:
                print(f"Erro: {caminho_zip} não é um arquivo ZIP válido.")
                os.remove(caminho_zip)

descompactar_arquivos()

Descompactado: ../Bronze_Layer\Atracacao_2021.zip -> ../Silver_Layer\Atracacao_2021
Descompactado: ../Bronze_Layer\Atracacao_2022.zip -> ../Silver_Layer\Atracacao_2022
Descompactado: ../Bronze_Layer\Atracacao_2023.zip -> ../Silver_Layer\Atracacao_2023
Descompactado: ../Bronze_Layer\CargaConteinerizada_2021.zip -> ../Silver_Layer\CargaConteinerizada_2021
Descompactado: ../Bronze_Layer\CargaConteinerizada_2022.zip -> ../Silver_Layer\CargaConteinerizada_2022
Descompactado: ../Bronze_Layer\CargaConteinerizada_2023.zip -> ../Silver_Layer\CargaConteinerizada_2023
Descompactado: ../Bronze_Layer\Carga_2021.zip -> ../Silver_Layer\Carga_2021
Descompactado: ../Bronze_Layer\Carga_2022.zip -> ../Silver_Layer\Carga_2022
Descompactado: ../Bronze_Layer\Carga_2023.zip -> ../Silver_Layer\Carga_2023


União dos arquivos em variáveis ("Atracacao", "Carga", "CargaConteinerizada")

In [6]:
input_dir = "../Silver_Layer"

arquivos = ["Atracacao", "Carga", "CargaConteinerizada"]
anos = ["2021", "2022", "2023"]

atracacao = None
carga = None
carga_cont = None

def carregar_dados(arquivo):
    input_files = [f"{input_dir}/{arquivo}_{ano}" for ano in anos]
    
    existing_files = []
    for folder in input_files:
        if os.path.exists(folder):
            files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".txt")]
            existing_files.extend(files)
    
    if not existing_files:
        print(f"{arquivo} não encontrado. Indo para o próximo...")
        return None
    
    dfs = [pd.read_csv(f, delimiter=";", header=0) for f in existing_files]
    return pd.concat(dfs, ignore_index=True) if dfs else None

atracacao = carregar_dados("Atracacao")
carga = carregar_dados("Carga")
carga_cont = carregar_dados("CargaConteinerizada")

C:\Users\clebe\AppData\Local\Temp\ipykernel_2736\2132257145.py:23: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, delimiter=";", header=0) for f in existing_files]
C:\Users\clebe\AppData\Local\Temp\ipykernel_2736\2132257145.py:23: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, delimiter=";", header=0) for f in existing_files]
C:\Users\clebe\AppData\Local\Temp\ipykernel_2736\2132257145.py:23: DtypeWarning: Columns (12,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, delimiter=";", header=0) for f in existing_files]
C:\Users\clebe\AppData\Local\Temp\ipykernel_2736\2132257145.py:23: DtypeWarning: Columns (12,17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f, delimiter=";", header=0) for f in existing_files]
C:\Users\clebe\AppData\

### Tratamento dos dados

Removendo valores NaN

In [7]:
atracacao_tratado = atracacao.fillna("")
carga_tratado = carga.fillna("")
carga_cont_tratado = carga_cont.fillna("")

Removendo acentuação e cedilha

In [8]:
def remover_acentos(df):
    return df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)

atracacao_tratado = remover_acentos(atracacao_tratado)
carga_tratado = remover_acentos(carga_tratado)
carga_cont_tratado = remover_acentos(carga_cont_tratado)

C:\Users\clebe\AppData\Local\Temp\ipykernel_2736\185886060.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: unidecode(x) if isinstance(x, str) else x)


Normalizando os nomes das colunas

In [9]:
def normalizar_nomes_colunas(df):
    df.columns = [
        unidecode(col).lower().replace(" ", "_") for col in df.columns
    ]
    return df

atracacao_tratado = normalizar_nomes_colunas(atracacao_tratado)
carga_tratado = normalizar_nomes_colunas(carga_tratado)
carga_cont_tratado = normalizar_nomes_colunas(carga_cont_tratado)

### Tratamento individual

Tabela atracação

In [10]:
atracacao_tratado.head(2)

,idatracacao,cdtup,idberco,berco,porto_atracacao,coordenadas,apelido_instalacao_portuaria,complexo_portuario,tipo_da_autoridade_portuaria,data_atracacao,...,flagmcoperacaoatracacao,terminal,municipio,uf,sguf,regiao_geografica,regiao_hidrografica,instalacao_portuaria_em_rio,no_da_capitania,no_do_imo
0,1206398,BRMCZ,MCZ0005,Berco 5,Maceio,"-35.726388,-9.683056000000001",,Maceio,Porto Organizado,10/12/2021 16:30:00,...,1,Cais do AA-12,Maceio,Alagoas,AL,Nordeste,,Nao,,
1,1176757,BRREC,RECA-05,RECA-05,Recife,"-34.868332,-8.053888000000001",Arrecife dos Navios,Suape - Recife,Porto Organizado,19/07/2021 08:15:00,...,1,Cais Publico,Recife,Pernambuco,PE,Nordeste,,Nao,,


In [11]:
atracacao_tratado["data_atracacao"] = pd.to_datetime(
    atracacao_tratado["data_atracacao"], format="%d/%m/%Y %H:%M:%S")

atracacao_tratado["data_chegada"] = pd.to_datetime(
    atracacao_tratado["data_chegada"], format="%d/%m/%Y %H:%M:%S")

atracacao_tratado["data_desatracacao"] = pd.to_datetime(
    atracacao_tratado["data_desatracacao"], format="%d/%m/%Y %H:%M:%S")

atracacao_tratado["data_inicio_operacao"] = pd.to_datetime(
    atracacao_tratado["data_inicio_operacao"], format="%d/%m/%Y %H:%M:%S")

atracacao_tratado["data_termino_operacao"] = pd.to_datetime(
    atracacao_tratado["data_termino_operacao"], format="%d/%m/%Y %H:%M:%S")

In [12]:
atracacao_tratado[["longitude", "latitude"]] = atracacao_tratado["coordenadas"].str.split(",", expand=True)
atracacao_tratado[["longitude", "latitude"]] = atracacao_tratado[["longitude", "latitude"]]

In [13]:
atracacao_tratado.head(2)

,idatracacao,cdtup,idberco,berco,porto_atracacao,coordenadas,apelido_instalacao_portuaria,complexo_portuario,tipo_da_autoridade_portuaria,data_atracacao,...,municipio,uf,sguf,regiao_geografica,regiao_hidrografica,instalacao_portuaria_em_rio,no_da_capitania,no_do_imo,longitude,latitude
0,1206398,BRMCZ,MCZ0005,Berco 5,Maceio,"-35.726388,-9.683056000000001",,Maceio,Porto Organizado,2021-12-10 16:30:00,...,Maceio,Alagoas,AL,Nordeste,,Nao,,,-35.726388,-9.683056000000001
1,1176757,BRREC,RECA-05,RECA-05,Recife,"-34.868332,-8.053888000000001",Arrecife dos Navios,Suape - Recife,Porto Organizado,2021-07-19 08:15:00,...,Recife,Pernambuco,PE,Nordeste,,Nao,,,-34.868332,-8.053888000000001


Tabela carga

In [14]:
carga_tratado.head(2)

,idcarga,idatracacao,origem,destino,cdmercadoria,tipo_operacao_da_carga,carga_geral_acondicionamento,conteinerestado,tipo_navegacao,flagautorizacao,...,percurso_transporte_em_vias_interiores,percurso_transporte_interiores,stnaturezacarga,stsh2,stsh4,natureza_da_carga,sentido,teu,qtcarga,vlpesocargabruta
0,27502716,1126626,BR200,BRBEL,2710,Interior,,,Interior,S,...,Interior de percurso nao identificado,Navegacao Interior,Exclusivo,Exclusivo,Exclusivo,Granel Liquido e Gasoso,Desembarcados,0,0,"2476,46"
1,27502733,1126639,BRBEL,BRMCP,2710,Interior,,,Interior,S,...,,,Exclusivo,Compartilhado,Compartilhado,Granel Liquido e Gasoso,Embarcados,0,0,"264,5"


In [15]:
carga_tratado["vlpesocargabruta"] = carga_tratado["vlpesocargabruta"].str.replace(",", ".").astype(float)
carga_tratado["teu"] = carga_tratado["teu"].str.replace(",", ".").astype(float)

Tabela carga conteineirizada

In [16]:
carga_cont.head(2)

,IDCarga,CDMercadoriaConteinerizada,VLPesoCargaConteinerizada
0,27566326,45G0,0
1,27566327,45G0,0


### Salvando os dados

In [17]:
atracacao_tratado.to_csv("../Silver_Layer/atracacao_tratado.csv", sep=";", index=False, encoding="utf-8")

carga_tratado.to_csv("../Silver_Layer/carga_tratado.csv", sep=";", index=False, encoding="utf-8")

carga_cont_tratado.to_csv("../Silver_Layer/carga_cont_tratado.csv", sep=";", index=False, encoding="utf-8")
